In [1]:
# SparkSQL
#SQL을 원하는 이유
#익숙한 언어인 SQL로 데이터를 분석하고 싶어서
#맵리듀스보다 빠르고 편함
#정형화된 데이터의 경우 높은 최적화를 구현 가능

#Spark DataFrame을 이용한 불리언데이터 처리#
#불리언 구문 - and, or, true, false

#pyspark의  jupyter notebook에서 SparkSession객체 생성하기
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Test").config("spark.executor.instances", "1").config("spark.executor.cores", "1").config("spark.executor.memory", "2g").config("spark.driver.memory", "2g").getOrCreate()



In [2]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/home/lab13/data/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")



root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365).select("InvoiceNo", "Description").show(5, False)  
#줄바꿀 때 백슬레시 필요 
#결과 집합 Truncating 하지 못하도록 False 인자 사용

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [6]:
#불리언 표현식을 사용하는 경우 모든 표현식을 and()로 묶어 차례대로 필터 적용해야 함
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()


+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [7]:
#Spark DataFrame을 이용한 수치데이터 처리#
from pyspark.sql.functions import  pow #지수만큼 컬럼의 값을 제곱

df.selectExpr(
  "CustomerId",
  "(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)


from pyspark.sql.functions import lit, round, bround  #반올림, 내림
df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [8]:
from pyspark.sql.functions import corr  #피어슨 상관계수 계산 
df.stat.corr("Quantity", "UnitPrice") # -0.04112314436835551
df.select(corr("Quantity", "UnitPrice")).show()
 
df.describe().show() #컬럼에 대한 집계, 평균, 표준편차, 최솟값, 최대값 계산(콘솔 확인용)




+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|          

In [9]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max

colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError) # 백분위수를 정확하게 계산

df.stat.crosstab("StockCode", "Quantity").show()   #교차 테이블(표) 생성
df.stat.freqItems(["StockCode", "Quantity"]).show() 



+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [10]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)  #모든 로우에 ID값 추가

from pyspark.sql.functions import  rand, randn  #난수 생성


+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
+-----------------------------+
only showing top 2 rows



In [11]:
#Spark DataFrame을 이용한 문자열데이터 처리#
from pyspark.sql.functions import initcap
from pyspark.sql.functions import lower, upper
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import translate
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import instr


In [12]:
#Spark DataFrame을 이용한 날짜데이터 처리#
#스파크는 자바의 날짜와 타임스탬프를 사용해서 표준체계를 따름
from pyspark.sql.functions import current_date, current_timestamp   
#오늘 날짜와 현재 타임스태프값 반환
dateDF = spark.range(10).withColumn("today", current_date()).withColumn("now", current_timestamp()) 
dateDF.createOrReplaceTempView("dateTable")

In [13]:
#날짜 컬럼에 수치데이터 산술연산
from pyspark.sql.functions import date_add, date_sub  
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

###날짜 컬럼에 날짜데이터 산술연산
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7)).select(datediff(col("week_ago"), col("today"))).show(1)

dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end")).select(months_between(col("start"), col("end"))).show(1)

#날짜 형식의 문자열 데이터를 날짜 데이터로 변환
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01")).select(to_date(col("date"))).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-02-07|        2020-02-17|
+------------------+------------------+
only showing top 1 row

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
+---------------+
only showing top 1 row



In [14]:
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")


#날짜 형식의 문자열 데이터를 타임스태프 데이터로 변환
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()


+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [15]:
#Spark DataFrame을 이용한 null데이터 처리#

#여러 컬럼값중 null이 아닌 첫번째 값을 반환
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"), col("CustomerId"))).show()

#null컬럼 row 삭제
df.na.drop("all", subset=["StockCode", "InvoiceNo"])  ## any 옵션값 사용 가능

#null컬럼값을 다른 값으로 채움
df.na.fill("all", subset=["StockCode", "InvoiceNo"])

fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)

df.na.replace([""], ["UNKNOWN"], "Description")

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [16]:
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [ ]:
##############################################################################################################

In [18]:
## 복합 데이터 타입다루기
#구조체 - DataFrame내부의 DataFrame
from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
complexDF
complexDF.show()


+--------------------+
|             complex|
+--------------------+
|[WHITE HANGING HE...|
|[WHITE METAL LANT...|
|[CREAM CUPID HEAR...|
|[KNITTED UNION FL...|
|[RED WOOLLY HOTTI...|
|[SET 7 BABUSHKA N...|
|[GLASS STAR FROST...|
|[HAND WARMER UNIO...|
|[HAND WARMER RED ...|
|[ASSORTED COLOUR ...|
|[POPPY'S PLAYHOUS...|
|[POPPY'S PLAYHOUS...|
|[FELTCRAFT PRINCE...|
|[IVORY KNITTED MU...|
|[BOX OF 6 ASSORTE...|
|[BOX OF VINTAGE J...|
|[BOX OF VINTAGE A...|
|[HOME BUILDING BL...|
|[LOVE BUILDING BL...|
|[RECIPE BOX WITH ...|
+--------------------+
only showing top 20 rows



In [19]:
# 배열  생성 -split() 
from pyspark.sql.functions import split
df.select(split(col("Description"), " ")).show(2)


+---------------------+
|split(Description,  )|
+---------------------+
| [WHITE, HANGING, ...|
| [WHITE, METAL, LA...|
+---------------------+
only showing top 2 rows



In [20]:
df.select(split(col("Description"), " ").alias("array_col")).selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



In [21]:
from pyspark.sql.functions import size
df.select(size(split(col("Description"), " "))).show(2) # 5와 3 출력

+---------------------------+
|size(split(Description,  ))|
+---------------------------+
|                          5|
|                          3|
+---------------------------+
only showing top 2 rows



In [22]:
from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

+--------------------------------------------+
|array_contains(split(Description,  ), WHITE)|
+--------------------------------------------+
|                                        true|
|                                        true|
+--------------------------------------------+
only showing top 2 rows



In [24]:
from pyspark.sql.functions import split, explode

df.withColumn("splitted", split(col("Description"), " ")).withColumn("exploded", explode(col("splitted"))).select("Description", "InvoiceNo", "exploded").show(2)


+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
+--------------------+---------+--------+
only showing top 2 rows



In [25]:
#맵 생성
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).show(2)


+--------------------+
|         complex_map|
+--------------------+
|[WHITE HANGING HE...|
|[WHITE METAL LANT...|
+--------------------+
only showing top 2 rows



In [26]:
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).selectExpr("complex_map['WHITE METAL LANTERN']").show(2)

#Map의 키와 값을 다시 컬럼으로 변환
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).selectExpr("explode(complex_map)").show(2)


+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
+--------------------------------+
only showing top 2 rows

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



In [27]:
## 2000년대 초 전자상거래 활성화 
## 이전까진 이 기중 간의 천차만별의 데이터를 

#JSON 컬럼 생성 
jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")
jsonDF.show()

+--------------------+
|          jsonString|
+--------------------+
|{"myJSONKey" : {"...|
+--------------------+



In [28]:
#get_json_object, json_tuple함수로 JSON객체 조회
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]").alias("column"),
    json_tuple(col("jsonString"), "myJSONKey")).show(2)

+------+--------------------+
|column|                  c0|
+------+--------------------+
|     2|{"myJSONValue":[1...|
+------+--------------------+



In [29]:
#JSON객체를 문자열로 변환 to_json
#JSON형식의 문자열을 다시 JSON객체로 변환 from_json
# json != to_json
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct").select(col("myStruct")).show(2, False)
df.selectExpr("(InvoiceNo, Description) as myStruct").select(to_json(col("myStruct"))).show(2, False)




+--------------------------------------------+
|myStruct                                    |
+--------------------------------------------+
|[536365, WHITE HANGING HEART T-LIGHT HOLDER]|
|[536365, WHITE METAL LANTERN]               |
+--------------------------------------------+
only showing top 2 rows

+-------------------------------------------------------------------------+
|structstojson(myStruct)                                                  |
+-------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
+-------------------------------------------------------------------------+
only showing top 2 rows



In [30]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *
parseSchema = StructType((  
  StructField("InvoiceNo",StringType(),True),
  StructField("Description",StringType(),True)))
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")).alias("newJSON"))\
  .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2, False) #생략 최소화 

+--------------------------------------------+-------------------------------------------------------------------------+
|jsontostructs(newJSON)                      |newJSON                                                                  |
+--------------------------------------------+-------------------------------------------------------------------------+
|[536365, WHITE HANGING HEART T-LIGHT HOLDER]|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|[536365, WHITE METAL LANTERN]               |{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
+--------------------------------------------+-------------------------------------------------------------------------+
only showing top 2 rows



In [31]:
#스파크에서 사용자 정의함수 사용 가능하며, 반드시 등록후 사용해야 함
udfExampleDF = spark.range(5).toDF("num")
def power3(double_value):
    return double_value ** 3     #함수 정의

power3(2.0)   #함수 호출


8.0

In [32]:

from pyspark.sql.functions import udf
power3udf = udf(power3)   #스파크에서 사용자 정의함수 등록


 # 등록된 사용자 정의 함수 사용
 # spark버전 충돌로 인하여 인지만 하고 넘어가기 
from pyspark.sql.functions import col
udfExampleDF.select(power3udf(col("num")))
udfExampleDF.select(power3udf(col("num"))).show(2) 

Py4JJavaError: An error occurred while calling o420.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 249, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.6 than that in driver 3.7, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:81)
	at org.apache.spark.sql.execution.python.PythonUDFRunne

In [ ]:
##########################################################################################################################################

In [33]:
### chapter7. 집계연산 사용해보기 
### 7.1. 집계함수부터

#retail 데이터 전체 다 불러오기(2010년 뿐만아니다)
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/home/lab13/data/retail_data/all/*.csv").coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable") 

In [34]:
from pyspark.sql.functions import count
df.select(count("StockCode")).show() # 541909

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [35]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show() # 4070

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [36]:
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show() # 3364

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [37]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()


+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [38]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [39]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 5176450

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [40]:
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show() # 29310

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [41]:
from pyspark.sql.functions import sum, count, avg, expr
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases")).selectExpr("total_purchases/total_transactions", "avg_purchases",    "mean_purchases").show()


+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [42]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
  stddev_pop("Quantity"), stddev_samp("Quantity")).show()


+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660923| 47559.39140929892|  218.08095663447835|   218.08115785023455|
+-----------------+------------------+--------------------+---------------------+



In [43]:
##비대칭도와 첨도 
from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610528376|119768.05495530753|
+--------------------+------------------+



In [44]:
##공분산과 상관관계
from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085640497E-4|             1052.7280543915997|            1052.7260778754955|
+-------------------------+-------------------------------+------------------------------+



In [45]:
##복합 데이터 타입의 집계 
from pyspark.sql.functions import collect_set, collect_list
df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [46]:
### 7.2 그룹화 
#그룹화 처리
from pyspark.sql.functions import count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show()

df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
  .show()


+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               

In [47]:
from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

In [48]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window\
  .partitionBy("CustomerId", "date")\
  .orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)


In [49]:
from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)


In [50]:
from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [51]:
from pyspark.sql.functions import col

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

In [54]:
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")
dfNoNull.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|U

In [53]:
#rollup 
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|   Netherlands|            97|
|2010-12-01|          EIRE|           243|
|2010-12-01|        France|           449|
|2010-12-01|     Australia|           107|
|2010-12-01|       Germany|           117|
|2010-12-01|          null|         26814|
|2010-12-01|United Kingdom|         23949|
|2010-12-01|        Norway|          1852|
|2010-12-02|       Germany|           146|
|2010-12-02|          null|         21023|
|2010-12-02|          EIRE|             4|
|2010-12-02|United Kingdom|         20873|
|2010-12-03|       Belgium|           528|
|2010-12-03|      Portugal|            65|
|2010-12-03|       Germany|           170|
|2010-12-03|   Switzerland|           110|
|2010-12-03|        France|           239|
|2010-12-03|          null|         14830|
|2010-12-03|         Spain|           400|
+----------

In [55]:
#cube
from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").show()


+----+--------------------+-------------+
|Date|             Country|sum(Quantity)|
+----+--------------------+-------------+
|null|               Japan|        25218|
|null|            Portugal|        16180|
|null|         Unspecified|         3300|
|null|             Germany|       117448|
|null|           Australia|        83653|
|null|                 RSA|          352|
|null|           Singapore|         5234|
|null|                 USA|         1034|
|null|              Cyprus|         6317|
|null|           Hong Kong|         4769|
|null|                null|      5176450|
|null|             Denmark|         8188|
|null|United Arab Emirates|          982|
|null|             Finland|        10666|
|null|      Czech Republic|          592|
|null|  European Community|          497|
|null|               Spain|        26824|
|null|     Channel Islands|         9479|
|null|             Lebanon|          386|
|null|              Norway|        19247|
+----+--------------------+-------

In [66]:
#grouping_id
#scarlar : import org.apache.spark.sql.functions.{grouping_id, sum, expr}
from pyspark.sql.functions import grouping_id ,sum ,expr
dfNoNull.cube("Date", "Country").agg(grouping_id(), sum("Quantity")).orderBy(desc("grouping_id()")).show()

+----+--------------------+-------------+-------------+
|Date|             Country|grouping_id()|sum(Quantity)|
+----+--------------------+-------------+-------------+
|null|                null|            3|      5176450|
|null|            Portugal|            2|        16180|
|null|             Finland|            2|        10666|
|null|           Australia|            2|        83653|
|null|           Singapore|            2|         5234|
|null|             Lebanon|            2|          386|
|null|                 RSA|            2|          352|
|null|               Japan|            2|        25218|
|null|               Spain|            2|        26824|
|null|     Channel Islands|            2|         9479|
|null|              Cyprus|            2|         6317|
|null|           Hong Kong|            2|         4769|
|null|         Unspecified|            2|         3300|
|null|             Germany|            2|       117448|
|null|      Czech Republic|            2|       

In [67]:
#pivot
pivoted = dfWithDate.groupBy("date").pivot("Country").sum() 
pivoted.printSchema() #스키마를 확인하고 검색조건 참조 


root
 |-- date: date (nullable = true)
 |-- Australia_sum(CAST(Quantity AS BIGINT)): long (nullable = true)
 |-- Australia_sum(UnitPrice): double (nullable = true)
 |-- Australia_sum(CAST(CustomerID AS BIGINT)): long (nullable = true)
 |-- Austria_sum(CAST(Quantity AS BIGINT)): long (nullable = true)
 |-- Austria_sum(UnitPrice): double (nullable = true)
 |-- Austria_sum(CAST(CustomerID AS BIGINT)): long (nullable = true)
 |-- Bahrain_sum(CAST(Quantity AS BIGINT)): long (nullable = true)
 |-- Bahrain_sum(UnitPrice): double (nullable = true)
 |-- Bahrain_sum(CAST(CustomerID AS BIGINT)): long (nullable = true)
 |-- Belgium_sum(CAST(Quantity AS BIGINT)): long (nullable = true)
 |-- Belgium_sum(UnitPrice): double (nullable = true)
 |-- Belgium_sum(CAST(CustomerID AS BIGINT)): long (nullable = true)
 |-- Brazil_sum(CAST(Quantity AS BIGINT)): long (nullable = true)
 |-- Brazil_sum(UnitPrice): double (nullable = true)
 |-- Brazil_sum(CAST(CustomerID AS BIGINT)): long (nullable = true)
 |-- Can

In [69]:
#pivoted.where("date > '2011-12-05'").select("date", "'USA_sum(Quantity)'").show  --- "'USA_sum(Quantity)'" 부재로 검색 안됨 
pivoted.where("date>'2011-12-05'").select("date","`USA_sum(UnitPrice)`").show()

+----------+------------------+
|      date|USA_sum(UnitPrice)|
+----------+------------------+
|2011-12-06|              null|
|2011-12-09|              null|
|2011-12-08|             13.75|
|2011-12-07|              null|
+----------+------------------+

